In [1]:
cd /content/drive/My Drive/Projects/Level sets/src

/content/drive/My Drive/Projects/Level sets/src


In [2]:
"""
Directory Structure: 

folder/
├── train_resnet.py
├── data_aug/
    ├── images_aug/ 
    ├── mask_aug/
    ├── out_aug/
    
"""

'\nDirectory Structure: \n\nfolder/\n├── train_resnet.py\n├── data_aug/\n    ├── images_aug/ \n    ├── mask_aug/\n    ├── out_aug/\n    \n'

In [3]:
!pip install segmentation_models_pytorch

     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 61kB 3.4MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=d9231dfd4ad8527e82654311b30f4d51030e0e668cc8ba7a29ff5b36afa16c76
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60962 sha256=8ad6e2aedfaa5bbc5fc238a674031c95c1bcb4d34ce2d43406b550d98a77cbfa
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built efficientnet-pytorch pretrainedmodels


In [0]:
import numpy as np 
from PIL import Image 
import scipy.ndimage as nd
import os
import cv2

import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchsummary import summary
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn
from tqdm import tqdm
import matplotlib.pyplot as plt

In [0]:
BASE_URL = "data_aug"   # main directory containing all folders 
BASE_DIR = "images_aug" # directory to fetch filenames from 

EPOCHS = 300
LEARNING_RATE = 0.0001

In [0]:
params = {
    'batch_size': 16,
    'shuffle': True,
    'num_workers': 6
    }

In [0]:
def bwdist(mask):
    return nd.distance_transform_edt(np.logical_not(mask))

def mask2levelSet(mask):
    return bwdist(mask) - bwdist(1 - mask)

In [0]:
class Dataset(Dataset):
  def __init__(self, base_dir, base_url, preprocessing_fn):
        'Initialization'
        self.base_url = base_url
        self.list_IDs = os.listdir(f"{base_url}/{base_dir}")
        self.preprocessing = preprocessing_fn

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        fileName = self.list_IDs[index]

        X_img = Image.open(f"{self.base_url}/images/{fileName}")    # loading image
        X_img = np.asarray(X_img)
        X_img = self.preprocessing(X_img).astype("float32")
        X_img = transforms.functional.to_tensor(X_img)                  # PIL image to Tensor 
        
        X_mask = Image.open(f"{self.base_url}/inits/{fileName}")
        X_mask = np.asarray(X_mask).astype("float32")                   # Image to np.array
        X_mask = transforms.functional.to_tensor(X_mask)                # converting to tensor 
        
        X = torch.cat((X_img, X_mask), dim=0)                           # X = [X_img, X_mask]
        

        y = Image.open(f"{self.base_url}/masks/{fileName}")           # loading image
        y = np.asarray(y)                                               # converting to array 
        y = y / np.max(y)                                               # y in [0, 1]
        y = 2*y-1                                                       # y in [-1, +1]
        y = y.astype("float32")                                         # changing data type
        Y = transforms.functional.to_tensor(y)                          # to tensor 
        
        return X, Y

In [0]:
model = smp.Unet('resnet50', encoder_weights='imagenet', in_channels = 4, classes = 1)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [0]:
preprocessing_fn = get_preprocessing_fn('resnet50', pretrained='imagenet')

In [0]:
summary(model, (4,256,256))

In [0]:
# unfreezing model layers 
for name, param in model.named_parameters():
    try:
        param.requires_grad = True
    except Exception as e: 
        print(f"{name}: {e}")

In [0]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# moving model to GPU if available
model.to(device)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [0]:
# defining data generator 
training_set = Dataset(BASE_DIR, BASE_URL, preprocessing_fn=preprocessing_fn)
training_generator = DataLoader(training_set, **params)

In [0]:
x,y = training_set[0]

In [0]:
# defining loss and optimizer 
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE) 

# training model 
train_losses = []
for e in range(EPOCHS):
    running_loss = 0
    for local_batch, local_labels in training_generator:
        inputs, labels = local_batch.to(device), local_labels.to(device)
        F = model(inputs)
        loss = criterion(2*F-1, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_losses.append(running_loss/len(training_generator))

    print(f"Epoch: {e+1}/{EPOCHS}.. Training Loss: {running_loss/len(training_generator):.6f}")

Epoch: 1/300.. Training Loss: 1.733986
Epoch: 2/300.. Training Loss: 0.998773
Epoch: 3/300.. Training Loss: 0.907478
Epoch: 4/300.. Training Loss: 0.826548
Epoch: 5/300.. Training Loss: 0.739906
Epoch: 6/300.. Training Loss: 0.632384
Epoch: 7/300.. Training Loss: 0.535367
Epoch: 8/300.. Training Loss: 0.446652
Epoch: 9/300.. Training Loss: 0.409288
Epoch: 10/300.. Training Loss: 0.337506
Epoch: 11/300.. Training Loss: 0.280213
Epoch: 12/300.. Training Loss: 0.232309
Epoch: 13/300.. Training Loss: 0.193206
Epoch: 14/300.. Training Loss: 0.186505
Epoch: 15/300.. Training Loss: 0.171602
Epoch: 16/300.. Training Loss: 0.148432
Epoch: 17/300.. Training Loss: 0.144777
Epoch: 18/300.. Training Loss: 0.130495
Epoch: 19/300.. Training Loss: 0.120847
Epoch: 20/300.. Training Loss: 0.113016
Epoch: 21/300.. Training Loss: 0.119827
Epoch: 22/300.. Training Loss: 0.101060
Epoch: 23/300.. Training Loss: 0.115338
Epoch: 24/300.. Training Loss: 0.138257
Epoch: 25/300.. Training Loss: 0.128606
Epoch: 26

KeyboardInterrupt: ignored

In [0]:
# saving model after training 
torch.save(model.state_dict(), f"{BASE_URL}/Unet_Model.pth" )

#Testing

In [5]:
model = smp.Unet('resnet34', encoder_weights='imagenet', in_channels = 4, classes = 1)
preprocessing_fn = get_preprocessing_fn('resnet34', pretrained='imagenet')

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth


In [0]:
BASE_URL = "../data/Aug_dataset"   # main directory containing all folders 
Experiemnt_URL = '../experiments/experiment_pytorch'
BASE_DIR = 'images'

In [7]:
# loading saved weights 
model.load_state_dict(torch.load(os.path.join(Experiemnt_URL, 'UNET_Model.pth')))

<All keys matched successfully>

In [8]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# moving model to GPU if available
model.to(device)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [0]:
testing_set = Dataset(BASE_DIR, BASE_URL, preprocessing_fn=preprocessing_fn)
testing_generator = DataLoader(testing_set, **params )

In [15]:
# creating output directory if it does not exist
if not os.path.exists(f"{Experiemnt_URL}/output"):
    os.makedirs(f"{Experiemnt_URL}/output")

i = 0
for X, Y in tqdm(testing_generator):
    # defining data generator 
    X = X.to(device)
    
    F = 2*model(X)-1
    F = F.cpu().detach().numpy()

    X = X.cpu().numpy()
    #Plotting the results
    figs = plt.figure(figsize=(16, 24))

    fig1 = figs.add_subplot(3,3,1)
    fig1.set_title('Input_Mask')
    fig1.imshow(X[0,-1,:,:], cmap='gray')

    fig2 = figs.add_subplot(3,3,2)
    fig2.set_title('Predicted')
    fig2.imshow(F[0,0,:,:], cmap='gray')

    fig3 = figs.add_subplot(3,3,3)
    fig3.set_title('GT Mask')
    fig3.imshow(Y[0,0,:,:], cmap='gray')

    plt.savefig(f"{Experiemnt_URL}/output/{i}.png")
    i+=1

Output hidden; open in https://colab.research.google.com to view.